In [2]:
import pandas as pd
import json

In [3]:
d_therapy = json.load(open("data/therapyByUser.json"))
d_therapy

{'uwHCq88eD7UtYRexpuWqxw': {'cbt': {'toyFactory': [1560442439, 1560342868],
   'associationSort': [1560443923, 1560342582, 1560258201],
   'lostRecipe': [1560442169, 1560341177, 1560258658]},
  'trt': {'theIsland': [1560444429, 1560341469, 1560341672],
   'simons': [1560342217],
   'constellations': [1560341129]},
  'relaxation': {'lullaby': [1560445371], 'coherentBreathing': [1560258533]},
  'knowledge': {},
  'questionnaire': {'residualInhibitionDiagnosis': [1560444709,
    1560445431,
    1560257471],
   'acouphenometry': [1560444932, 1560257346, 1560257563]},
  'residualInhibition': {'keepTheShape': [1560442633, 1560257901],
   'smoothieBar': [1560444558, 1560342434, 1560258014]}},
 'SD-WxvJXQoIPxSTWQRHUOQ': {'cbt': {'lostRecipe': [1560443036,
    1560350858,
    1560412958,
    1560325994,
    1560268572,
    1560099017,
    1560013287,
    1559926422,
    1559979859,
    1559807409,
    1559751895,
    1559720345,
    1559665641,
    1559590925,
    1559634309,
    1559408404,
  

In [17]:
data_activity = {"user":[],"therapy":[],"activity":[],"time":[]}
for userKey in d_therapy:
    user = d_therapy[userKey]
    for therapyKey in user:
        therapy = user[therapyKey]
        for activityKey in therapy:
            activity = therapy[activityKey]
            for time in activity:
                data_activity["user"].append(userKey)
                data_activity["therapy"].append(therapyKey)
                data_activity["activity"].append(activityKey)
                data_activity["time"].append(time)

df = pd.DataFrame(data_activity)
(df.groupby("user")["therapy"].count()>50).value_counts()

False    2057
True      129
Name: therapy, dtype: int64

In [32]:
time_sorted = df[(df["user"] == 'SD-WxvJXQoIPxSTWQRHUOQ') & (df["therapy"] == 'cbt') & (df["activity"] == 'lostRecipe')]["time"].sort_values()
if len(time_sorted) > 1:
    time_sorted = list(time_sorted)
    time_differencies = [(time_sorted[i+1]-time_sorted[i]) for i in range(len(time_sorted)-1)]
    print(sum(time_differencies)/len(time_differencies)/86400)

1.0288473915989158


In [37]:
!pip install progressbar2

In [41]:
from progressbar.bar import ProgressBar


In [42]:
df_frequency = use_frequencies(df)
df_frequency.head()

 65% (22695 of 34428) |###########       | Elapsed Time: 0:08:16 ETA:   2:11:50

KeyboardInterrupt: 

In [54]:
def detect_change_activity(df):
    resu = [0]
    n = df.shape[0]
    for i in range(n-1):
        row = df.iloc[[i]]
        next_row = df.iloc[[i+1]]
        if row["user"].values != next_row["user"].values:
            resu.append(i+1)
        elif row["activity"].values != next_row["activity"].values:
            resu.append(i+1)
    if resu[-1] != n:
        resu.append(n)
    return resu

In [78]:
row=df.iloc[[0]]
row["user"].values[0]

'uwHCq88eD7UtYRexpuWqxw'

In [57]:
changes = detect_change_activity(df)

In [59]:
len(changes)
changes[0:2]

[0, 2]

In [74]:
def frequencies_from_dataframe(df):
    changes = detect_change_activity(df)
    pbar = ProgressBar(min_value=0,max_value=df.shape[0]).start()
    i=0
    d_frequency = {}
    for i in range(len(changes)-1):
        start, end = changes[i], changes[i+1]-1
        if end - start > 1:
            time_sorted = df["time"].iloc[start:end].sort_values(ascending=False)
            time_sorted = list(time_sorted)
            time_differencies = [(time_sorted[i]-time_sorted[i+1]) for i in range(len(time_sorted)-1)]
            time_differencies.sort()
            median = time_differencies[len(time_differencies) // 2]
            d_frequency[start]= median/86400
        pbar.update(end)
    pbar.finish
    return d_frequency

In [75]:
df_frequencies = frequencies_from_dataframe(df)
df_frequencies

 97% (33557 of 34428) |################# | Elapsed Time: 0:00:00 ETA:   0:00:00

{2: 1.1729282407407406,
 5: 1.1688888888888889,
 8: 1.1916666666666667,
 15: 0.008356481481481482,
 18: 2.1711342592592593,
 23: 1.1819907407407408,
 26: 0.71875,
 109: 0.6668287037037037,
 208: 0.6637731481481481,
 316: 0.6336574074074074,
 449: 0.6194791666666667,
 599: 1.0089699074074074,
 610: 0.6637037037037037,
 698: 1.1679861111111112,
 744: 1.1235532407407407,
 795: 1.9292013888888888,
 810: 1.1353472222222223,
 829: 0.6391319444444444,
 953: 1.0710300925925926,
 989: 1.061087962962963,
 995: 0.0006365740740740741,
 1003: 1.0530324074074073,
 1014: 1.176412037037037,
 1093: 1.1574652777777779,
 1169: 1.0572569444444444,
 1263: 1.0154976851851851,
 1356: 1.1819444444444445,
 1436: 0.6104050925925926,
 1579: 1.9811921296296295,
 1617: 1.8291087962962962,
 1662: 3.912673611111111,
 1669: 7.308842592592592,
 1674: 7.309988425925926,
 1679: 1.0570833333333334,
 1740: 1.1886342592592594,
 1821: 0.11240740740740741,
 1826: 2.068773148148148,
 1844: 2.3678125,
 1866: 1.0331597222222222

In [79]:
def use_frequencies(df):
    d_frequency_row = frequencies_from_dataframe(df)
    d_frequency = {"user":[],"therapy":[],"activity":[],"frequency":[]}
    for row_id in d_frequency_row:
        row = df.iloc[[row_id]]
        d_frequency["user"].append(row["user"].values[0])
        d_frequency["therapy"].append(row["therapy"].values[0])
        d_frequency["activity"].append(row["activity"].values[0])
        d_frequency["frequency"].append(d_frequency_row[row_id])
    df_frequency = pd.DataFrame(d_frequency)
    return df_frequency

In [81]:
df_frequency = use_frequencies(df)
df_frequency.head(20)

 97% (33557 of 34428) |################# | Elapsed Time: 0:00:00 ETA:   0:00:00

,user,therapy,activity,frequency
0,uwHCq88eD7UtYRexpuWqxw,cbt,associationSort,1.172928
1,uwHCq88eD7UtYRexpuWqxw,cbt,lostRecipe,1.168889
2,uwHCq88eD7UtYRexpuWqxw,trt,theIsland,1.191667
3,uwHCq88eD7UtYRexpuWqxw,questionnaire,residualInhibitionDiagnosis,0.008356
4,uwHCq88eD7UtYRexpuWqxw,questionnaire,acouphenometry,2.171134
5,uwHCq88eD7UtYRexpuWqxw,residualInhibition,smoothieBar,1.181991
6,SD-WxvJXQoIPxSTWQRHUOQ,cbt,lostRecipe,0.718750
7,SD-WxvJXQoIPxSTWQRHUOQ,cbt,toyFactory,0.666829
8,SD-WxvJXQoIPxSTWQRHUOQ,cbt,associationSort,0.663773
9,SD-WxvJXQoIPxSTWQRHUOQ,trt,constellations,0.633657
